In [ ]:
# project: CSGO AutoAim (train)
# author: GY
# version: 1.0
# time: 2021.2.23
# descirption:

"""
This is a computer vision project based on paddleX and 
a computer game named CS:GO, which is used to detect the
enemies and shot automatically. It is implemented by deep
learning method Fast-RCNN and pyautogui lib. You can use 
it to learn or just for fun in the game (just like me), 
but never use it for cheating. Thanks for your cooperation!

"""


'\nThis is a computer vision project based on paddleX and \na computer game named CS:GO, which is used to detect the\nenemies and shot automatically. It is implemented by deep\nlearning method Fast-RCNN and pyautogui lib. You can use \nit to learn or just for fun in the game (just like me), \nbut never use it for cheating. Thanks for your cooperation!\n\n'

In [1]:
# install paddlex
! pip install paddlex -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple


In [ ]:
# import datasets zip
import zipfile
zf = zipfile.ZipFile("datasets_new.zip")
zf.extractall("train/")

In [2]:
# GPU setting
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

In [3]:
# data transform
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32),
])

In [ ]:
import os
import random


trainval_percent = 0.1
train_percent = 0.9
xmlfilepath = 'train/Annotations'
txtsavepath = 'train/ImageSets/Main'
total_xml = os.listdir(xmlfilepath)
num = len(total_xml)
list = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)
#os.makedirs('train/ImageSets/Main')
ftrainval = open('train/ImageSets/Main/trainval.txt', 'w')
ftest = open('train/ImageSets/Main/test.txt', 'w')
ftrain = open('train/ImageSets/Main/train.txt', 'w')
fval = open('train/ImageSets/Main/val.txt', 'w')

for i in list:
    name = total_xml[i][:-4]
    if i in trainval:
        ftrainval.write("JPG/"+name+".jpg " + "Annotations/"+name+".xml\n")
        if i in train:
            ftest.write("JPG/"+name+".jpg " + "Annotations/"+name+".xml\n")
        else:
            fval.write("JPG/"+name+".jpg " + "Annotations/"+name+".xml\n")
    else:
        ftrain.write("JPG/"+name+".jpg " + "Annotations/"+name+".xml\n")

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

In [4]:
# making train & valid datasets
import paddlex as pdx
train_dataset = pdx.datasets.VOCDetection(
    data_dir='train/',
    file_list='train/ImageSets/Main/train.txt',
    label_list='train/ImageSets/Main/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.VOCDetection(
    data_dir='train/',
    file_list='train/ImageSets/Main/val.txt',
    label_list='train/ImageSets/Main/labels.txt',
    transforms=eval_transforms)

2021-02-25 23:18:09 [INFO]	Starting to read file list from dataset...
2021-02-25 23:18:09 [INFO]	166 samples in file train/ImageSets/Main/train.txt
creating index...
index created!
2021-02-25 23:18:09 [INFO]	Starting to read file list from dataset...
2021-02-25 23:18:09 [INFO]	2 samples in file train/ImageSets/Main/val.txt
creating index...
index created!


In [6]:
# training
! rm -rf ~/log & rm -rf output/faster_rcnn_r50_fpn
! mkdir -p output/faster_rcnn_r50_fpn/vdl_log
! ln -s output/faster_rcnn_r50_fpn/vdl_log ~/log

decay_epoch=list(range(64))
num_classes = len(train_dataset.labels) + 1
model = pdx.det.FasterRCNN(num_classes=num_classes)
model.train(
    num_epochs=64,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.0001,
    lr_decay_epochs=decay_epoch[20:],
    lr_decay_gamma=0.95,
    save_interval_epochs=1,
    save_dir='output/faster_rcnn_r50_fpn',
    use_vdl=True)

2021-02-25 23:20:18,197-INFO: If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!


2021-02-25 23:20:18 [INFO]	Downloading ResNet50_cos_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_cos_pretrained.tar


100%|██████████| 100310/100310 [00:01<00:00, 57276.60KB/s]


2021-02-25 23:20:20 [INFO]	Decompressing output/faster_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.tar...
2021-02-25 23:20:24 [INFO]	Load pretrain weights from output/faster_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.


2021-02-25 23:20:24,382-WARNING: output/faster_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.pdparams not found, try to load model file saved with [ save_params, save_persistables, save_vars ]


...

2021-02-25 23:53:55 [INFO]	[EVAL] Finished, Epoch=62, bbox_map=68.686869 .
2021-02-25 23:53:59 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_62.
2021-02-25 23:53:59 [INFO]	Current evaluated best model in eval_dataset is epoch_10, bbox_map=83.33333333333334
2021-02-25 23:54:06 [INFO]	[TRAIN] Epoch=63/64, Step=2/41, loss=0.221013, loss_cls=0.114169, loss_bbox=0.094334, loss_rpn_cls=0.00534, loss_rpn_bbox=0.00717, lr=1.1e-05, time_each_step=0.67s, eta=0:1:8
2021-02-25 23:54:07 [INFO]	[TRAIN] Epoch=63/64, Step=4/41, loss=0.1792, loss_cls=0.094744, loss_bbox=0.077406, loss_rpn_cls=0.003593, loss_rpn_bbox=0.003457, lr=1.1e-05, time_each_step=0.68s, eta=0:1:8
2021-02-25 23:54:08 [INFO]	[TRAIN] Epoch=63/64, Step=6/41, loss=0.308334, loss_cls=0.157563, loss_bbox=0.127269, loss_rpn_cls=0.008393, loss_rpn_bbox=0.01511, lr=1.1e-05, time_each_step=0.7s, eta=0:1:7
2021-02-25 23:54:09 [INFO]	[TRAIN] Epoch=63/64, Step=8/41, loss=0.099867, loss_cls=0.057949, loss_bbox=0.034476, loss_rpn_cls=0.

100%|██████████| 2/2 [00:03<00:00,  1.62s/it]


2021-02-25 23:54:25 [INFO]	[EVAL] Finished, Epoch=63, bbox_map=61.111111 .
2021-02-25 23:54:30 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_63.
2021-02-25 23:54:30 [INFO]	Current evaluated best model in eval_dataset is epoch_10, bbox_map=83.33333333333334
2021-02-25 23:54:35 [INFO]	[TRAIN] Epoch=64/64, Step=1/41, loss=0.265464, loss_cls=0.122991, loss_bbox=0.110821, loss_rpn_cls=0.024255, loss_rpn_bbox=0.007397, lr=1e-05, time_each_step=0.65s, eta=0:0:33
2021-02-25 23:54:37 [INFO]	[TRAIN] Epoch=64/64, Step=3/41, loss=0.161469, loss_cls=0.084753, loss_bbox=0.05702, loss_rpn_cls=0.005896, loss_rpn_bbox=0.0138, lr=1e-05, time_each_step=0.68s, eta=0:0:33
2021-02-25 23:54:38 [INFO]	[TRAIN] Epoch=64/64, Step=5/41, loss=0.208325, loss_cls=0.113235, loss_bbox=0.082658, loss_rpn_cls=0.007585, loss_rpn_bbox=0.004848, lr=1e-05, time_each_step=0.69s, eta=0:0:32
2021-02-25 23:54:39 [INFO]	[TRAIN] Epoch=64/64, Step=7/41, loss=0.176578, loss_cls=0.087884, loss_bbox=0.077577, loss_rpn_cls=0.

100%|██████████| 2/2 [00:04<00:00,  2.43s/it]


2021-02-25 23:54:57 [INFO]	[EVAL] Finished, Epoch=64, bbox_map=69.318182 .
2021-02-25 23:55:02 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_64.
2021-02-25 23:55:02 [INFO]	Current evaluated best model in eval_dataset is epoch_10, bbox_map=83.33333333333334


In [ ]:
# single image predict
image_name = 'yaoping_data/JPEGImages/Error6_7803_12_32_131922.jpg'
result = model.predict(image_name)
pdx.det.visualize(image_name, result, threshold=0.5, save_dir='output/resimg')

In [10]:
# valid sets predict
import os
import paddlex as pdx
file = open("train/ImageSets/Main/train.txt") 
model = pdx.load_model('output/faster_rcnn_r50_fpn/epoch_64')
for line in file.readlines():  
     print(line)
     line=line.strip('\n').split(' ')[0]
     image_name = 'train/' + line
     result = model.predict(image_name)
     pdx.det.visualize(image_name, result, threshold=0.5, save_dir='output/resimg_train')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 